Wahlpflichtach Künstliche Intelligenz I: Praktikum 

---

# 06 - Erstes Data Wrangling mit Pandas

- [Das Pandas series-Objekt](#Das-Pandas-series-Objekt)
- [Das Pandas DataFrame-Objekt](#Das-Pandas-DataFrame-Objekt)
- [Das Pandas Index-Objekt](#Das-Pandas-Index-Objekt)
- [Indizierung, Auswahl und Zuweisung von Daten](#Indizierung,-Auswahl-und-Zuweisung-von-Daten)
- [Lesen von Serien und DataFrames](#Lesen-von-Serien-und-DataFrames)
- [Ufuncs und Aggregation](#Ufuncs-und-Aggregation)
- [Group-By](#Group-By)
- [Aggregate,-filter,-transform,-apply](#Aggregate,-filter,-transform,-apply)
- [Plots speichern](#Plots-speichern)

Pandas ist eine Bibliothek für schnelle und effiziente Berechnungen auf großen Datenmengen. Wie in Numpy sind auch in Pandas viele Operationen vektorisiert und damit effizient und schnell.


Pandas ist ein neueres Paket, das auf NumPy aufbaut und eine effiziente Implementierung eines DataFrames bietet. DataFrames sind im Wesentlichen mehrdimensionale Arrays mit angehängten Zeilen- und Spaltenbezeichnungen und oft mit heterogenen Typen und/oder fehlenden Daten. Pandas bietet nicht nur eine bequeme Speicherschnittstelle für beschriftete Daten, sondern implementiert auch eine Reihe leistungsfähiger Datenoperationen, die Benutzern sowohl von Datenbank-Frameworks (-> relationale Algebra) als auch von Tabellenkalkulationsprogrammen vertraut sind.

Wie wir gesehen haben, bietet die ndarray-Datenstruktur von NumPy wesentliche Eigenschaften für die Art von sauberen, gut organisierten Daten, die typischerweise bei numerischen Berechnungsaufgaben vorkommen. Während sie diesen Zweck sehr gut erfüllt, werden ihre Grenzen deutlich, wenn wir mehr Flexibilität benötigen (z. B. das Anhängen von Beschriftungen an Daten, die Arbeit mit fehlenden Daten usw.) und wenn wir Operationen versuchen, die sich nicht gut auf die elementweise Übertragung abbilden lassen (z. B. Gruppierungen, Pivots usw.), was jeweils ein wichtiger Bestandteil der Analyse der weniger strukturierten Daten ist, die in vielen Formen in der Welt um uns herum vorhanden sind. Pandas und insbesondere seine Series- und DataFrame-Objekte bauen auf der NumPy-Array-Struktur auf und bieten einen effizienten Zugriff auf diese Art von "Datenverarbeitungs"-Aufgaben, die einen Großteil der Zeit eines Datenwissenschaftlers in Anspruch nehmen.

So wie wir numpy normalerweise als np importieren, importieren wir pandas unter dem Alias von pd. Wir importieren auch numpy, weil wir es bei der Verwendung von pandas oft brauchen werden:

In [ ]:
import numpy as np
import pandas as pd

## Das Pandas `series`-Objekt

Eine Pandas-`series` ist ein eindimensionales Array mit indizierten Daten. Es kann wie folgt aus einer Liste oder einem Array erzeugt werden:

In [ ]:
# Series with missing values 
data = pd.Series([0.25, 0.5, np.NaN, 1.0])
data

In [ ]:
type(data)

In [ ]:
data.values, type(data.values)

Der Index ist ein Array-ähnliches Objekt vom Typ `pd.Index`:

In [ ]:
data.index, type(data.index), list(data.index)

Wie bei einer Liste oder einem NumPy-Array kann auf die Daten über den zugehörigen Index über die bekannte Python-Schreibweise mit eckigen Klammern zugegriffen werden:

In [ ]:
data[1:3]

In [ ]:
type(data[1])

In [ ]:
print(dir(data))

### Serie als verallgemeinertes NumPy-Array

Nach dem, was wir bisher gesehen haben, mag es so aussehen, als sei das Series-Objekt grundsätzlich austauschbar mit einem eindimensionalen NumPy-Array. Der wesentliche Unterschied ist das Vorhandensein des Indexes: Während das Numpy-Array einen implizit definierten Integer-Index hat, der für den Zugriff auf die Werte verwendet wird, hat die Pandas-Serie einen explizit definierten Index, der mit den Werten verknüpft ist.

Diese explizite Indexdefinition verleiht dem Series-Objekt zusätzliche Fähigkeiten. Zum Beispiel muss der Index kein Integer sein, sondern kann aus Werten jedes beliebigen Typs bestehen. Wenn wir wollen, können wir zum Beispiel Zeichenketten als Index verwenden:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'd', 'c'])
data

In [ ]:
data.index = list("AbCD")
data

In [ ]:
data["b"] == data[1] == data.values[1]

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[3, 7, 2, 4])
data

In [ ]:
data.index

Wenn ein expliziter Index vorhanden ist, wird er bevorzugt! (*Solange wir nicht slicen!*)

In [ ]:
data[3]

In [ ]:
data[1:3]

In [ ]:
type(data[3])

Beachten Sie, dass explizite Indizes nicht eindeutig sein müssen!

In [ ]:
d2 = pd.concat([data, data])
d2

In [ ]:
d2[7]

In [ ]:
d2[7] = 2

In [ ]:
d2

### Serie als spezialisiertes Dictionary

Auf diese Weise können Sie sich eine Pandas-Serie ein bisschen wie eine Spezialisierung eines Python-Dictionarys vorstellen. Ein Dictionary ist eine Struktur, die beliebige Schlüssel auf eine Menge von beliebigen Werten abbildet, und eine Series ist eine Struktur, die typisierte Schlüssel auf eine Menge von typisierten Werten abbildet. Diese Typisierung ist wichtig: So wie der typspezifische kompilierte Code hinter einem NumPy-Array es für bestimmte Operationen effizienter macht als eine Python-Liste, macht die Typinformation einer Pandas-Serie sie für bestimmte Operationen viel effizienter als Python-Dictionary.

Die Serie-als-Dictionary-Analogie kann noch deutlicher gemacht werden, indem ein Serie-Objekt direkt aus einem Python-Dictionary konstruiert wird:

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

In [ ]:
population['Texas']

Im Gegensatz zu einem Dictionary unterstützt die Serie jedoch auch Array-ähnliche Operationen wie z. B. Slicing:

In [ ]:
population['California':'Illinois']

Beachten Sie, dass Illinois inklusive ist!

### Erstellen von Serienobjekten

Daten können ein Skalar sein:

In [ ]:
pd.Series(5, index=[100, 200, 300])

Daten können ein Dictionary sein:

In [ ]:
ser = pd.Series({2:'a', 1:'b', 3:'c'})
ser

In [ ]:
ser.to_dict()

In [ ]:
list(ser.values)

## Das Pandas DataFrame-Objekt

Die nächste grundlegende Struktur in Pandas ist der DataFrame. Wie das Series-Objekt kann es entweder als eine Verallgemeinerung eines NumPy-Arrays oder als eine Spezialisierung eines Python-Dictionarys betrachtet werden. Wir werden nun einen Blick auf jede dieser Perspektiven werfen.

### DataFrame als verallgemeinertes NumPy-Array

Wenn eine Serie ein Analogon zu einem eindimensionalen Array mit flexiblen Indizes ist, ist ein DataFrame ein Analogon zu einem zweidimensionalen Array mit flexiblen Zeilenindizes und flexiblen Spaltennamen. Genauso wie Sie sich ein zweidimensionales Array als eine geordnete Folge von ausgerichteten eindimensionalen Spalten vorstellen können, können Sie sich einen DataFrame als eine Folge von ausgerichteten Series-Objekten vorstellen. Mit "ausgerichtet" meinen wir hier, dass sie denselben Index haben.



Um dies zu demonstrieren, konstruieren wir zunächst eine neue Serie, die die Fläche jedes der fünf im vorherigen Abschnitt besprochenen Zustände auflistet:

In [ ]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

Nun, da wir dies zusammen mit der Bevölkerungsreihe von vorher haben, können wir ein Dictionary verwenden, um ein einzelnes zweidimensionales Objekt zu konstruieren, das diese Informationen enthält:

In [ ]:
states = pd.DataFrame({'population': population,
                       'area': area,
                       'country': 'USA'})
states

In [ ]:
print(states.dtypes)

Das sieht aus wie ein verallgemeinertes Dictionary! Die Keys sind die Namen der Staaten, und die Werte sind wie eine Liste `[area, country, population]`.

In [ ]:
states.sort_values(by="population", ascending=False)

In [ ]:
states['population'], type(states['population'])

Ermittelt die "Keys" (Indizes) des DataFrames, in dem "Bevölkerung" seinen Maximalwert hat:

In [ ]:
states["population"].idxmax()

Gibt die Serie am angegebenen Index zurück:

In [ ]:
states.loc[states["population"].idxmax()]

Gibt das Maximum für jede einzelne Spalte zurück:

In [ ]:
states.max()

In [ ]:
states['California']

In [ ]:
states.loc['California']

In [ ]:
states.index

In [ ]:
states.columns

In [ ]:
states.values

In [ ]:
type(states.values)

Man kann sich den DataFrame als eine Verallgemeinerung eines zweidimensionalen NumPy-Arrays vorstellen, bei dem sowohl die Zeilen als auch die Spalten einen verallgemeinerten Index für den Zugriff auf die Daten haben.

### DataFrame als spezialisiertes Dictionary

In ähnlicher Weise können wir uns einen DataFrame auch als eine Spezialisierung eines Dictionarys vorstellen. Wo ein Dictionary einen Key auf einen Value abbildet, bildet ein DataFrame einen Spaltennamen auf eine Serie von Spaltendaten ab. Wenn Sie z. B. nach dem Attribut "area" fragen, erhalten Sie das Objekt "Series", das die Bereiche enthält, die wir zuvor gesehen haben.

Beachten Sie, dass die Indizierung eines DataFrames mit eckigen Klammern die *Spalte* erhält!

In [ ]:
states["area"]

In [ ]:
type(states["area"])

### Erstellen von DataFrame-Objekten

Ein Pandas DataFrame kann auf verschiedene Arten konstruiert werden:

#### Aus einem einzelnen Series-Objekt

Ein DataFrame ist eine Sammlung von Series-Objekten, und ein einspaltiger DataFrame kann aus einer einzigen Series konstruiert werden:

In [ ]:
population

In [ ]:
pd.DataFrame(population, columns=['population'])

#### Aus mehreren Series-Objekten

In [ ]:
s1 = pd.Series(['100', '200', 'python', '300.12', '400'])
s2 = pd.Series(['10', '20', 'php', '30.12', '40'])
df = pd.concat([s1, s2], axis='columns')
df

Beachten Sie, dass viele Funktionen in Pandas das Argument `Achse` nehmen - in diesem Fall können Sie zwischen 0/`Index` und 1/`Spalten` wählen. Wenn Sie explizit sein wollen, empfehle ich die Verwendung der String-Version!

#### Aus einer Liste von Dictionaries 

Eine beliebige Liste von Dictionaries kann zu einem DataFrame gemacht werden. Wir werden eine einfache List Comprehension verwenden, um einige Daten zu erzeugen. Auch wenn einige Keys im Dictionary fehlen, wird Pandas diese mit NaN (d.h. "not a number") Werten auffüllen:

In [ ]:
df = pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}], index=["first_dict", "second_dict"])
df

Da jede einzelne Spalte einen konsistenten D-Typ haben muss und np.NaN ein Float ist, werden einige der Zahlen in Floats umgewandelt:

In [ ]:
df['a']

In [ ]:
df['b']

In [ ]:
type(np.NaN)

In [ ]:
df.dtypes

Wenn wir die Zeilen erhalten wollten, müsste Pandas die Zahlen explizit erzwingen: 

In [ ]:
df

In [ ]:
df.loc['first_dict']

#### Aus einem zweidimensionalen NumPy-Array

Aus einem zweidimensionalen Array von Daten können wir einen DataFrame mit den angegebenen Spalten- und Indexnamen erstellen. Wenn nichts angegeben wird, wird ein ganzzahliger Index für jede Spalte verwendet:

In [ ]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

**Wiederholen Sie das bis jetzt Gelernte in Bezug auf die grundlegenden Objektypen von Pandas für sich!**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

**Erzeugen Sie einen DataFrame aus dem angegebenen Dictionary sowie der `qualifies`-Liste, mit den angegebenen Indizes:**
```python
exam_data = {
    'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'],
    'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
    'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    }

qualifies = ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
```

## Das Pandas Index-Objekt

Wir haben hier gesehen, dass sowohl das Series- als auch das DataFrame-Objekt einen expliziten Index enthalten, mit dem Sie Daten referenzieren und verändern können. Dieses Index-Objekt ist eine interessante Struktur für sich, die man sich als unveränderliches Array vorstellen kann:

In [ ]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

In [ ]:
ind[0] = 1

In [ ]:
sr = pd.Series(0, index=ind)
sr

Index-Objekte haben einen Namen:

In [ ]:
ind.names = ['indexx']
ind

In [ ]:
sr = pd.Series(np.zeros_like(ind), index=ind)
sr

In [ ]:
df = pd.DataFrame(np.zeros_like(ind), index=ind, columns=['first'])
df

In [ ]:
df.index.names = [None]
df

Index-Objekte haben auch viele der Attribute, die von NumPy-Arrays bekannt sind:

In [ ]:
ind.size, ind.shape, ind.ndim, ind.dtype

Während die Betrachtung von Indizes als unveränderliche Liste natürlich ist, erlauben Indizes auch Mengenoperationen:

In [ ]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [ ]:
indA & indB

In [ ]:
indA | indB

In [ ]:
indA ^ indB

## Indizierung, Auswahl und Zuweisung von Daten

Aus dem Numpy Notebook kennen wir bereits Indizierung, Slicing, Maskierung und Fancy Indexing:

In [ ]:
a = np.arange(16).reshape(4,4)
a

Nimmt die Werte der zweiten und vierten Spalte, die durch 3 teilbar sind:

In [ ]:
a[:, [1, 3]][a[:, [1, 3]] % 3 == 0]

Hier werden wir uns ähnliche Möglichkeiten des Zugriffs auf und der Änderung von Werten in Pandas Series- und DataFrame-Objekten ansehen. Die entsprechenden Muster in Pandas sind denen von Numpy sehr ähnlich, obwohl es ein paar Besonderheiten zu beachten gibt.

Wir beginnen mit dem einfachen Fall des eindimensionalen Series-Objekts und gehen dann zu dem etwas komplizierteren zweidimensionalen DataFrame-Objekt über.

### Datenauswahl in Serien

Wie wir im vorherigen Abschnitt gesehen haben, verhält sich ein Series-Objekt in vielerlei Hinsicht wie ein eindimensionales NumPy-Array und in vielerlei Hinsicht wie ein Standard-Python-Dictionary. Wenn wir uns diese beiden sich überschneidenden Analogien vor Augen halten, wird es uns helfen, die Muster der Datenindizierung und -auswahl in diesen Arrays zu verstehen.

#### Serie als Dictionary

Wie ein Dictionary bietet das Series-Objekt eine Abbildung von einer Sammlung von Keys auf eine Sammlung von Values, was bedeutet, dass die meisten der entsprechenden Funktionen genauso gut für sie funktionieren:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

In [ ]:
data.__contains__('b')

In [ ]:
'b' in data

In [ ]:
np.array_equal(data.keys(), data.index)

In [ ]:
data

In [ ]:
list(data.items())

In [ ]:
data['e'] = 1.25
data

#### Serie als eindimensionales Array

Series baut auf dieser wörterbuchähnlichen Schnittstelle auf und bietet die Auswahl von Elementen im Array-Stil über die gleichen grundlegenden Mechanismen wie NumPy-Arrays - also Slices, Maskierung und ausgefallene Indexierung. Beispiele hierfür sind die folgenden:

* Slicing durch expliziten Index

In [ ]:
data['a':'c']

* Slicing durch impliziten Integer-Index

(Beachten Sie, dass beim Slicing mit einem expliziten Index (d. h. data['a':'c']) der letzte Index in das Slice eingeschlossen wird, während beim Slicing mit einem impliziten Index (d. h. data[0:2]) der letzte Index vom Slice ausgeschlossen wird.)

In [ ]:
data[0:2]

In [ ]:
(data > 0.3) & (data < 0.8)

* Maskierung

In [ ]:
data[(data > 0.3) & (data < 0.8)]

* Fancy indexing

In [ ]:
data[['a', 'e']]

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[1, 2, 3, 4])
data

In [ ]:
data[1:3]

**Wenn Ihre Serie einen expliziten Integer-Index hat, verwendet eine Indizierungsoperation wie `data[1]` die expliziten Indizes, während eine Slicing-Operation wie `data[1:3]` den impliziten Index im Python-Stil verwendet.**

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

Expliziter Index bei Indizierung:

In [ ]:
data[1]

Impliziter Index beim Slicen:

In [ ]:
data[1:3]

Das `loc`-Attribut ermöglicht eine Indizierung und Slicing, die *immer* auf den expliziten Index verweist:

In [ ]:
data.loc[1]

In [ ]:
data.loc[1:3]

Beachten Sie, dass `loc` beim Slicen Index-Fehler auslösen kann, aber nicht muss:

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

In [ ]:
data.loc['a':'z']

In [ ]:
data.loc[3:10]

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data.loc[3:10]

In [ ]:
data.loc['a':'z']

Das Attribut `iloc` ermöglicht eine Indizierung und Slicing, die immer auf den impliziten Index im Python-Stil verweist:

In [ ]:
data.iloc[1]

In [ ]:
data.iloc[1:3]

Bitte, ersparen Sie sich den Schmerz und seien Sie immer explizit über das, was Sie tun -- benutzen Sie immer ``.loc`` und ``.iloc``

**Explicit is better than implicit.**

Die Aussage ist **nicht**, dass explizite Indizes besser sind als implizite, sondern dass Sie explizit sein sollten, was Sie tun.

### Nachtrag: Indizierung

Aus [der Doku][1]: In früheren Versionen funktionierte die Verwendung von .loc[list-of-labels], solange mindestens 1 der Schlüssel gefunden wurde (andernfalls wurde ein KeyError ausgelöst). Dieses Verhalten ist jetzt veraltet!

[1]: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

In [ ]:
s = pd.Series([1, 2, 3])
s

In [ ]:
s.loc[[1, 2, 3]]

Stattdessen sollten Sie sich für [`reindex`][1] entscheiden, das die Serie/DF an einen neuen Index mit optionaler Fülllogik anpasst.

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html

In [ ]:
s.reindex([1, 2, 3])

### Datenselektion in DataFrames

Erinnern Sie sich daran, dass sich ein DataFrame in vielerlei Hinsicht wie ein zweidimensionales oder strukturiertes Array verhält und in anderer Hinsicht wie ein Wörterbuch mit Reihenstrukturen, die denselben Index haben. Diese Analogien können hilfreich sein, wenn wir die Datenauswahl innerhalb dieser Struktur untersuchen.

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop, 'Country':'USA'})
data

Beachten Sie, dass wir, wenn wir einen DataFrame indizieren, die **Spalte** indizieren!!! Die Indizierung im Stil eines Dictionarys ergibt eine Serie.

In [ ]:
print(type(data["area"]))
data["area"]

Wir können auch dereferenzieren, allerdings führt das zu Seiteneffekten, auch wenn das eigentlich auch eine Methode ist.

In [ ]:
data.area

In [ ]:
type(data.values)

Mit diesem Bild im Kopf können viele bekannte Array-ähnliche Beobachtungen am DataFrame selbst durchgeführt werden. Zum Beispiel können wir den gesamten DataFrame transponieren, um Zeilen und Spalten zu vertauschen:

In [ ]:
data.T

Für die Indexierung im Array-Stil verwendet Pandas wieder die bereits erwähnten loc- und iloc-Indexer. Mit dem iloc-Indexer können wir das zugrundeliegende Array so indizieren, als wäre es ein einfaches NumPy-Array (unter Verwendung des impliziten Python-Style-Index), **aber der DataFrame-Index und die Spaltenbeschriftungen werden im Ergebnis** beibehalten (Indizierung des zugrunde liegenden Numpy-Arrays):

In [ ]:
data.values[:3, :2]

In [ ]:
data.iloc[:3, :2]

In [ ]:
data

In [ ]:
data.loc[:'Illinois', :'pop']

In [ ]:
data.loc[:,['area','pop']]

So bekommen wir eine Reihe:

In [ ]:
data.loc["California", :]

Hinzufügen einer neuen Spalte (mit Vektor-Berechnungen):

In [ ]:
data['density'] = data['pop'] / data['area']
data

Wir können eine Maskierung mit Fancy Indexing kombinieren:

In [ ]:
data.loc[data.density > 100, ['pop', 'density']]

Wenn Sie explizite und implizite Indizierung kombinieren wollen, müssen Sie sie verketten:

In [ ]:
data.iloc[1:4].loc[:, ['pop', 'density']]

**Während sich die Indizierung auf Spalten bezieht, bezieht sich Slicing auf Zeilen:**

In [ ]:
data['area']

In [ ]:
data['Florida':'Illinois']

Auch hier gilt: Seien Sie bei der Indizierung lieber explizit, um sich eine Menge Verwirrung zu ersparen.

In [ ]:
data['area':'pop']

In [ ]:
data.loc[:, 'area':'pop']

Schneller Zugriff auf einen einzelnen Teilnehmer mit **at**:

In [ ]:
%%timeit
data.loc['Florida', 'pop']

In [ ]:
%%timeit
data.at['Florida', 'pop']

In [ ]:
data

### Reindexing

In [ ]:
index = ['Firefox', 'Chrome', 'Safari', 'IE10', 'Konqueror']
df = pd.DataFrame({'http_status': [200, 200, 404, 404, 301],
                  'response_time': [0.04, 0.02, 0.07, 0.08, 1.0]},
                  index=index)
df

In [ ]:
new_index = ['Safari', 'Iceweasel', 'Comodo Dragon', 'IE10', 'Chrome']
df.reindex(new_index)

In [ ]:
df.reindex(columns=['http_status', 'user_agent'])

**Renaming indices** 

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3, 4], "b": [2, 5, 7, 8]})
df = df.rename(mapper={'b': 'c'}, axis='columns')
df

### Boolean Indexing

In [ ]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df['E'] = ["one", "two", "three"] * 2
df

In [ ]:
df['A'] > 0.5

In [ ]:
df[df['A'] > 0]

In [ ]:
df.query('A > 0')

[Alternative Syntax](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) für `query`.

In [ ]:
df['E'].isin(['one','two'])

In [ ]:
df[df['E'].isin(['one','two'])] = np.NaN
df

In [ ]:
pd.isna(df)

In [ ]:
pd.isna(df).any(axis=1)

In [ ]:
df[~df.isna().any(axis=1)]

In [ ]:
df.dropna(how="any")

**Machen Sie sich jetzt vertieft mit der Indizierung, der Auswahl und der Zuweisung von Daten in Pandas vertraut!**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

**Schreiben Sie ein Pandas-Snippet, um die Namen und Punktzahlen der Personen, bei denen die Anzahl der Versuche in der Prüfung größer als 2 ist, als Dictionary zu erhalten.**
```python
exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael',
                      'Matthew', 'Laura', 'Kevin', 'Jonas'],
    'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
    'attempts' : [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

df = pd.DataFrame(exam_data , index=labels)
df
```


### Datenzuweisung

In [ ]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                  index=['Portland', 'Berkeley'])
df

### Spalten zuweisen

In [ ]:
df['country'] = 'USA'
df

In [ ]:
df['temp_c'] <= 18

In [ ]:
df['too_cold'] = df['temp_c'] <= 18
df

Diese arbeiten jedoch in-place. Um die Werte einem neuen Datenframe zuzuweisen, verwenden Sie `assign`:

In [ ]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                  index=['Portland', 'Berkeley'])
df

In [ ]:
df2 = df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32)
df2

In [ ]:
#vectorized version:
df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)

In [ ]:
df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
          temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)

Mehrere Zuordnungen gleichzeitig sind ebenfalls möglich:

In [ ]:
df

### Zeilenzuordnungen:

In [ ]:
df.loc['Berkeley', 'temp_c'] = 26.0
df

In [ ]:
type(df.loc['Portland'])

In [ ]:
df.loc['Portland'] = pd.Series({'temp_c': 99})
df

In [ ]:
df.loc['Osnabruck', 'temp_c'] = 18
df

In [ ]:
df = pd.concat([df, df])
df

In [ ]:
df.loc['Osnabruck', 'temp_c'] = 25
df

In [ ]:
df.loc['Osnabruck'] = pd.Series({'temp_c': 99})
df

In [ ]:
type(df.loc['Osnabruck'])

In [ ]:
np.where(df.index == 'Osnabruck')

In [ ]:
df.iloc[np.where(df.index == 'Osnabruck')[0][0]]

In [ ]:
df.iloc[np.where(df.index == 'Osnabruck')[0][0]] = pd.Series({'temp_c': 99})
df

### Multi-Indizierung

Obwohl Pandas Objekte zur Verfügung stellt, die nativ mit drei- und vierdimensionalen Daten umgehen können, ist es in der Praxis weitaus verbreiteter, eine "hierarchische Indizierung" (auch bekannt als "Multi-Indizierung") zu verwenden, um mehrere Indexebenen innerhalb eines einzigen Indexes einzubinden. Auf diese Weise können höherdimensionale Daten kompakt innerhalb der bekannten eindimensionalen Series- und zweidimensionalen DataFrame-Objekte dargestellt werden.

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

In [ ]:
index = pd.MultiIndex.from_tuples(index)
index

In [ ]:
index.names = ['state', 'year']

In [ ]:
pop = pop.reindex(index)
pop

In [ ]:
pop['California', 2000], pop['California', 2010]

In [ ]:
pop.iloc[0]

In [ ]:
pop.iloc[1]

### Multi-Indizierung

Obwohl Pandas Objekte zur Verfügung stellt, die nativ mit drei- und vierdimensionalen Daten umgehen können, ist es in der Praxis weitaus verbreiteter, eine "hierarchische Indizierung" (auch bekannt als "Multi-Indizierung") zu verwenden, um mehrere Indexebenen innerhalb eines einzigen Indexes einzubinden. Auf diese Weise können höherdimensionale Daten kompakt innerhalb der bekannten eindimensionalen Series- und zweidimensionalen DataFrame-Objekte dargestellt werden.

In [ ]:
pop.unstack()

In [ ]:
index.names = [None, None]
pop = pop.reindex(index)
pop

In [ ]:
pop.unstack()

In [ ]:
pop.index.names = [None, None]
pop.unstack().T

In [ ]:
popdf = pop.unstack(level=0)
popdf

In [ ]:
popdf.stack()

### Index setzen und zurücksetzen

Eine weitere Möglichkeit, hierarchische Daten neu zu ordnen, besteht darin, die Indexbezeichnungen in Spalten umzuwandeln; dies kann mit der Methode ``reset_index`` erreicht werden.

Der Aufruf dieser Methode auf dem Populationsverzeichnis führt zu einem ``DataFrame`` mit einer *Staat* und *Jahr* Spalte, die die Informationen enthält, die vorher im Index waren.

Der Übersichtlichkeit halber können wir optional den Namen der Daten für die Spaltendarstellung angeben:

In [ ]:
pop

In [ ]:
pop.index.names = ['state', 'year']
print(type(pop))
pop

In [ ]:
pop_flat = pop.reset_index(name='population')
pop_flat

Wenn man in der realen Welt mit Daten arbeitet, sehen die rohen Eingabedaten oft so aus und es ist nützlich, einen ``MultiIndex`` aus den Spaltenwerten zu bilden.

Das kann man mit der Methode ``set_index`` des ``DataFrame`` machen, die einen mehrfach indizierten ``DataFrame`` zurückgibt:

In [ ]:
pop_df = pop_flat.set_index(['state', 'year'])
pop_df

In [ ]:
pop_df.rename_axis([None, None])

In [ ]:
asdf = pop_df.rename_axis([None, None]).unstack()
asdf

In [ ]:
asdf.columns

In [ ]:
asdf["area"] = 999
asdf

In [ ]:
asdf.columns

In [ ]:
print(type(asdf["area"]))
asdf["area"]

In [ ]:
print(type(asdf["population"]))
asdf["population"]

In [ ]:
pop_flat

In [ ]:
pop_df2 = pop_flat.set_index('state').rename_axis(None)
pop_df2

In [ ]:
pop_df

In [ ]:
pop_df.reset_index()

## Lesen von Serien und DataFrames

In [ ]:
df = pd.read_csv("data/06/Pokemon.csv")

Stellen Sie sich vor, jemand gibt Ihnen einen zufälligen Datensatz. Sie kennen nichts von dessen Inhalt. Was sind die ersten Schritte, die Sie tun?

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df["Type 1"].value_counts()

In [ ]:
df["Legendary"].value_counts()

Wenn Sie mit csvs arbeiten, sollten Sie unbedingt darauf achten, ob Sie die erste Spalte als Index-Spalte haben wollen!

In [ ]:
df = pd.read_csv("data/06/Pokemon.csv", index_col=0)
df.tail()

In [ ]:
df.reset_index().tail()

In [ ]:
df.reset_index().drop_duplicates(subset="#").tail()

In [ ]:
df = df[df['Name'] != 'Volcanion']
df.tail()

In [ ]:
no_duplicates = df.reset_index().drop_duplicates(subset="#").reset_index().drop("index", axis=1)  
no_duplicates.tail()

In [ ]:
no_duplicates.set_index("#").to_csv('data/06/Pokemon_no_duplicates.csv')
#no_duplicates.to_excel('Pokemon_no_duplicates.xlsx', sheet_name='Sheet1')

In [ ]:
gen_one = no_duplicates[no_duplicates["Generation"] == 1].set_index("#")
gen_one.tail()

In [ ]:
first_gen_dict = gen_one["Name"].to_dict()

In [ ]:
[str(key)+" : "+str(val) for index, (key, val) in enumerate(first_gen_dict.items()) if index < 9]

**Dokumentation!**

Es gibt wirklich sehr viele Argumente für diese Funktion, passend für alle Ihre Bedürfnisse!

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

## Ufuncs und Aggregation

### Aggregation in Pandas

Aggregationen sind Funktionen, bei denen eine oder mehrere Dimensionen von Daten auf einen einzigen Wert zusammengefasst werden, wie z.B. die `max`, `sum` oder `mean`- Funktionen.

Stat-Operationen *exkludieren* im Allgemeinen fehlende Daten.

#### Für Serien

In [ ]:
a = np.arange(7)
ser = pd.Series(a**2, index=a)
ser

In [ ]:
ser.sum()

In [ ]:
ser.mean()

In [ ]:
ser.median()

In [ ]:
ser.min()

In [ ]:
ser.max()

#### Für DataFrames

In [ ]:
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

In [ ]:
df.mean()

In [ ]:
df.mean(axis=0)

In [ ]:
df.mean(axis='rows')

In [ ]:
df.mean(axis=1)

Die folgende Tabelle fasst einige weitere eingebaute Pandas-Aggregationen zusammen:

| Aggregation | Beschreibung |
|--------------------------|---------------------------------|
| ``count()`` | Gesamtzahl der Elemente |
| ``first()``, ``last()`` | Erstes und letztes Element |
| ``mean()``, ``median()`` | Mittelwert und Median |
| ``min()``, ``max()`` | Minimum und Maximum |
| ``std()``, ``var()`` | Standardabweichung und Varianz |
| ``mad()`` | Mittlere absolute Abweichung |
| ``prod()`` | Produkt aller Positionen |
| ``sum()`` | Summe aller Elemente |

Dies sind alles Methoden der Objekte ``DataFrame`` und ``Series``.

### Ufuncs

Ufuncs kennen wir bereits aus Numpy: Das sind vektorisierte Funktionen, die alle Werte eines Arrays gleichzeitig verändern. 

Pandas macht dasselbe, mit einem netten Twist: Für unäre Operationen wie Negation und trigonometrische Funktionen werden diese Ufuncs die Index- und Spaltenbeschriftungen in der Ausgabe *beibehalten*, und für binäre Operationen wie Addition und Multiplikation wird Pandas die Indizes automatisch *ausrichten*, wenn die Objekte an die Ufunc übergeben werden.

Das bedeutet, dass das Beibehalten des Kontexts von Daten und das Kombinieren von Daten aus verschiedenen Quellen - beides potenziell fehleranfällige Aufgaben mit rohen NumPy-Arrays - mit Pandas im Wesentlichen narrensicher werden.

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

In [ ]:
np.exp(df)

### Ufuncs: Index-Ausrichtung

Bei binären Operationen auf zwei ``Series``- oder ``DataFrame``-Objekten richtet Pandas die Indizes während der Ausführung der Operation aus.

Dies ist sehr praktisch, wenn man mit unvollständigen Daten arbeitet.

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

In [ ]:
population

In [ ]:
area & population

In [ ]:
area / population

In [ ]:
"divide" in dir(pd.DataFrame)

In [ ]:
popdens = area.divide(population, fill_value=0)
popdens

In [ ]:
popdens = popdens.replace([np.inf, -np.inf], np.nan)
popdens.dropna()

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

In [ ]:
B = pd.DataFrame(rng.randint(0, 20, (3, 3)),
                 columns=list('ABC'))
B

In [ ]:
A+B

In [ ]:
A.add(B, fill_value=0)

#### More Index-Alignment

In [ ]:
df = pd.DataFrame({'a': np.random.randint(3, size=10)}, index=np.arange(1, 20, 2))
df

Fügen wir eine neue Spalte zu diesem DataFrame hinzu!

In [ ]:
tmp = pd.Series([0]*len(df.index))
tmp

In [ ]:
#df['new'] = tmp   #changes the original one
df.assign(new=tmp) #creates a copy

In [ ]:
df.index | tmp.index

In [ ]:
old_aligned, new_aligned = df.align(tmp, axis=0)
old_aligned

In [ ]:
new_aligned

In [ ]:
old_aligned.assign(new=new_aligned)

In [ ]:
tmp = pd.Series([0]*len(df.index), index=df.index)
tmp

In [ ]:
df['new'] = tmp
df

Wenn Sie mehr als eine Operation (ufunc/Aggregation) anwenden wollen, verwenden Sie `agg()`:

In [ ]:
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                  columns=['A', 'B', 'C'])
df

In [ ]:
df.agg(['sum', 'min'])

Sie können auch verschiedene Aggregationen für verschiedene Spalten verwenden: 

In [ ]:
df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})

Das funktioniert auch für ufuncs:

In [ ]:
df.agg({'A' : 'exp', 'B' : [np.exp, 'sqrt']})

#### apply()

Während einige ufuncs (wie cumsum oder exp) von Pandas vordefiniert sind, kann die Methode `apply` verwendet werden, um eine beliebige Funktion auf alle Elemente einer Serie oder eines DataFrame anzuwenden.

In [ ]:
a = np.arange(7)
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

In [ ]:
df.cumsum()

In [ ]:
df.apply(np.cumsum)

In [ ]:
df["A_cumsum"] = df.cumsum()["A"]
df["B_cumsum"] = df.apply(np.cumsum)["B"]
df

Mit Lambda-Funktionen können wir `apply` mit beliebigen Funktionen kombinieren. Beachten Sie, dass das Argument der Funktion immer eine ganze Spalte des Datensatzes ist.

In [ ]:
df.apply(lambda x: print(x, end='\n\n'))

In [ ]:
df

In [ ]:
df['A'] + 1

In [ ]:
df.apply(lambda x: x+1)

In [ ]:
def my_more_complex_func(ser):
    res = []
    for elem in ser:
        print(elem if elem > 16 else -elem)
        res.append(elem if elem > 16 else -elem)
    return res

In [ ]:
df.apply(my_more_complex_func)

In [ ]:
df

In [ ]:
df.apply(lambda x: x.max() - x.min())

In [ ]:
df["A"].apply(lambda x: print(x))

In [ ]:
df["A_normed"] = df["A"].apply(lambda x: x/df["A"].max())
df

Wir können sogar Dictionaries mit der apply-Funktion verwenden!

In [ ]:
z_moves = {"Normal": "Breakneck Blitz", "Fighting": "All-Out Pummeling", "Flying": "Supersonic Skystrike", "Poison": "Acid Downpour", "Ground": "Tectonic Rage", "Rock": "Continental Crush", "Bug": "Savage Spin-Out", "Ghost": "Never-Ending Nightmare",
"Steel": "Corkscrew Crash", "Fire": "Inferno Overdrive", "Water": "Hydro Vortex", "Grass": "Bloom Doom", "Electric": "Gigavolt Havoc", "Psychic": "Shattered Psyche", "Ice": "Subzero Slammer", "Dragon": "Devastating Drake", "Dark": "Black Hole Eclipse", "Fairy": "Twinkle Tackle"}
df = pd.read_csv("data/06/Pokemon.csv")
df.head()

In [ ]:
df["Z-Move"] = df["Type 1"].apply(lambda x:z_moves[x])
df.head()

Mit `apply` können wir auch eine Liste von Reihen in einen DataFrame umwandeln, indem wir die einzelnen Spalten zu Reihen machen:

In [ ]:
s = pd.Series([ ['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']]) 
print(type(s))
s

In [ ]:
pd.Series([1, 2, 3])

In [ ]:
df = s.apply(pd.Series)
print(type(df))
df

**Notiz zur Geschwindigkeit:**

Laut ([1]) ist `apply()` doppelt so schnell wie das Durchlaufen von `iterrows()` einer df und 8 mal so schnell wie das Durchlaufen von Python-Listen.

Beachten Sie jedoch, dass `apply()` zwar viel schneller beim Schleifen über die Zeilen Ihres DataFrame/Series ist (indem es eine Reihe von internen Optimierungen nutzt, wie z.B. die Verwendung von Iteratoren in Cython), aber immer noch von Natur aus über Zeilen schleift. Was auch immer Sie anwenden, Sie führen es immer noch einmal für jede Zeile aus. Wo immer Sie also vektorisierte Ufuncs verwenden können, tun Sie das - das ist viel optimierter und parallelisierter - für ([1]) führte der Austausch der Haversinus-Abstandsformel mit ihrem vektorisierten Gegenstück zu einer 50-fachen Zeitersparnis!

[1]: https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6 

**Wiederholen Sie jetzt die verschiedenen Funktionen und die Möglichkeiten der Aggregation in Pandas!**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

**Schreiben Sie ein Pandas-Programm, um Reihen von Listen in eine Reihe zu konvertieren.**
```python
s = pd.Series([ ['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']])
s
```

## Group-By

### Split-Apply-Combine

Während einfache Operationen bereits von Pandas vordefiniert sind, können benutzerdefinierte Aggregationen und Operationen über **group-by** durchgeführt werden. Die Group-by-Operation kann mit den folgenden Schritten beschrieben werden:

* **Split:** Aufteilung der Daten in Gruppen basierend auf einigen Kriterien (Aufteilung und Gruppierung in Abhängigkeit vom Wert eines Schlüssels)
* **Apply:** Anwendung einer Funktion auf jede Gruppe unabhängig (Aggregation, Transformation, Filterung, ...)
* **Combine:** Zusammenfassen der Ergebnisse in einer Datenstruktur

Ein typisches Beispiel, bei dem die *Anwendung* eine Summierungsaggregation ist, ist hier dargestellt:

In [ ]:
tmp = np.array([list("ABCABC"), np.arange(1,7)]).T
tmp

In [ ]:
df = pd.DataFrame(tmp, columns=["key", "data"])
df["data"] = pd.to_numeric(df["data"])
df

In [ ]:
df.groupby("key")

Beachten Sie, dass nicht ein Satz von "DataFrames" zurückgegeben wird, sondern ein "DataFrameGroupBy"-Objekt. In diesem Objekt liegt die Magie: Sie können es sich als eine spezielle Ansicht der `DataFrames` vorstellen, die bereit ist, in den Gruppen zu graben, aber keine tatsächliche Berechnung durchführt, bis die Aggregation angewendet wird. Dieser "Lazy-Evaluation"-Ansatz bedeutet, dass gängige Aggregate sehr effizient und für den Benutzer nahezu transparent implementiert werden können.

Um ein Ergebnis zu erzeugen, können wir ein Aggregat auf dieses "DataFrameGroupBy"-Objekt anwenden, das die entsprechenden Anwendungs-/Verbindungsschritte durchführt, um das gewünschte Ergebnis zu erzeugen:

In [ ]:
df.groupby("key").sum().reset_index()

Wir können die Spaltenindizierung genau wie bei einem normalen DataFrame durchführen:

In [ ]:
df.groupby("key")["data"].sum()

### Iteration über Gruppen

Das ``GroupBy``-Objekt unterstützt die direkte Iteration über die Gruppen und gibt jede Gruppe als ``Series`` oder ``DataFrame`` zurück:

In [ ]:
df

In [ ]:
for (key, _) in df.groupby("key"):
    print(key)
    
print()
for (_, group) in df.groupby("key"):
    print(group, "\n")

In [ ]:
pkm = pd.read_csv('data/06/Pokemon.csv')
pkm.groupby('Generation')['Total'].mean()

### Dispatch-Methoden

Jede Methode, die nicht explizit durch das ``GroupBy``-Objekt implementiert ist, wird durchgereicht und auf den Gruppen aufgerufen, egal ob es sich um ``DataFrame``- oder ``Series``-Objekte handelt.
Zum Beispiel können Sie die Methode ``describe()`` von ``DataFrame`` verwenden, um eine Reihe von Aggregationen durchzuführen, die jede Gruppe in den Daten beschreiben:

In [ ]:
df.describe()

In [ ]:
df.groupby("key").describe()

In [ ]:
df = pd.read_csv("data/06/Pokemon_no_duplicates.csv", index_col=0)
df.groupby('Generation')["Name"].nunique()

**Wiederholen Sie jetzt die verschiedenen Funktionen und die Möglichkeiten der Aggregation in Pandas!**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

**Der gegebene Datensatz enthält eine Spalte `Region` sowie eine Spalte `Pop. Density`. Schreiben Sie ein Snippet, das als Argument den Datenrahmen mit allen Ländern nimmt und eine `Series` zurückgibt, die Regionen auf die durchschnittliche Bevölkerungsdichte ihrer Länder abbildet.**
```python
countries = pd.read_csv('data/06/countries.csv', index_col=0)
countries.head()
```

## Aggregate, filter, transform, apply

Bisher haben wir uns auf das Aggregieren für die combine-Operation konzentriert, aber es gibt noch mehr Möglichkeiten.
Insbesondere ``GroupBy``-Objekte haben ``aggregate()``, ``filter()``, ``transform()`` und ``apply()``-Methoden, die eine Vielzahl nützlicher Operationen effizient implementieren, bevor die gruppierten Daten kombiniert werden.

Für die Zwecke der folgenden Unterabschnitte werden wir diesen ``DataFrame`` verwenden:

In [ ]:
def create_df():
    rng = np.random.RandomState(0)
    df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'data1': range(6),
                       'data2': rng.randint(0, 10, 6)},
                       columns = ['key', 'data1', 'data2'])
    return df
    
df = create_df()
df

### Aggregation

Während wir bereits einige *Aggregations-Funktionen* verwendet haben, ist die Funktion `aggregate` die explizite Version davon.  
Sie kann eine Zeichenkette, eine Funktion oder eine Liste davon nehmen und alle Aggregate auf einmal berechnen.

In [ ]:
df

In [ ]:
df.groupby('key').agg(['min', np.median, max])

Ein weiteres nützliches Vorgehen ist die Übergabe eines Wörterbuchs, das (alte) Spaltennamen auf Operationen abbildet, die auf diese Spalte angewendet werden sollen:

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

In [ ]:
df.groupby('key').aggregate({'data1': np.sum,
                             'data2': lambda x: np.std(x, ddof=1)})

### Benannte Aggregation

Um spaltenspezifische Aggregation mit Kontrolle über die *Ausgabespaltennamen* zu unterstützen, akzeptiert Pandas sogenannte benannte Aggregation in `GroupBy.agg()`, wobei:

* Die Schlüsselwörter sind die Namen der Ausgabespalten
* Die Werte sind Tupel, deren erstes Element die auszuwählende Spalte und das zweite Element die Aggregation ist, die auf diese Spalte angewendet werden soll. 
    * Alternativ können Sie das NamedAgg-Tupel pandas.NamedAgg mit den Feldern ['column', 'aggfunc'] verwenden, um deutlicher zu machen, was die Argumente sind. 

In [ ]:
animals = pd.DataFrame({'kind': ['cat', 'dog', 'cat', 'dog'],
                        'height': [9.1, 6.0, 9.5, 34.0],
                        'weight': [7.9, 7.5, 9.9, 198.0]})
animals

In [ ]:
assert int(pd.__version__[0]) >= 1, 'Your version of pandas is too old for this!'

In [ ]:
animals.groupby("kind").agg(
       min_height = pd.NamedAgg(column='height', aggfunc='min'),
       max_height = pd.NamedAgg(column='height', aggfunc='max'),
       average_weight = pd.NamedAgg(column='weight', aggfunc=np.mean),
    )

### Filtern

Mit einer Filterung können Sie Daten basierend auf den Gruppeneigenschaften ausschließen.
Zum Beispiel könnten wir alle Gruppen behalten wollen, in denen die Standardabweichung größer als ein kritischer Wert ist:

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

df

In [ ]:
df.groupby('key').std()

Beachten Sie, dass dies kein Aggregat ist - das Ergebnis hat die Form des ursprünglichen DataFrame, nur dass bestimmte Zeilen weggelassen wurden!

In [ ]:
df.groupby('key').filter(filter_func)

### Die Methode apply()

Mit der Methode ``apply()`` können Sie eine beliebige Funktion auf die Gruppenergebnisse anwenden.
Die Funktion sollte einen ``DataFrame`` annehmen und entweder ein Pandas Objekt (z.B. ``DataFrame``, ``Series``) oder einen Skalar zurückgeben; die Kombinationsoperation wird auf den Typ der zurückgegebenen Ausgabe zugeschnitten.

Erinnern Sie sich zunächst an unser ``apply()`` von vorhin:

In [ ]:
create_df()

In [ ]:
df = create_df()
df["data1"] = df["data1"].apply(lambda x: x/df["data1"].max())
df

Beachten Sie, dass ``groupby`` nur eine *Ansicht des ursprünglichen DataFrame* zurückgibt. Hier ist eine ``Anwendung()``, die die (gruppierte) erste Spalte durch die Summe der (gruppierten) zweiten normalisiert:

In [ ]:
import warnings; warnings.filterwarnings('ignore')
try:
    del newdf
except:
    pass

In [ ]:
df = create_df()
sums = df.groupby('key')["data2"].sum()
print(sums, '\n\n\n')
for key, group in df.groupby('key'):
    group["data1"] /= sums[key]
    try:
        newdf = newdf.append(group) #appending to dataframes is bad style!
    except:
        newdf = group.copy()
    print(newdf, '\n')

newdf

In [ ]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

df

In [ ]:
df.groupby('key').apply(norm_by_data2)

### Angeben des Split-Schlüssels

In den zuvor vorgestellten einfachen Beispielen haben wir den ``DataFrame`` auf einen einzelnen Spaltennamen aufgeteilt.
Dies ist nur eine von vielen Optionen, mit denen die Gruppen definiert werden können, und wir werden hier einige andere Optionen für die Gruppenspezifikation durchgehen.

### Eine Liste, ein Array, eine Serie oder ein Index, der die Gruppierungsschlüssel liefert

Der Schlüssel kann eine beliebige Serie oder Liste mit einer Länge sein, die der des ``DataFrame`` entspricht. Zum Beispiel:

In [ ]:
df

In [ ]:
L = [0, 1, 0, 1, 2, 0]
df.groupby(L).sum()

### Ein Dictionary oder eine Serie, die den Index auf die Gruppe abbildet

Eine andere Methode ist die Bereitstellung eines Dictionarys, das Indexwerte auf die Gruppenschlüssel abbildet:

In [ ]:
df2 = df.set_index('key')
df2

In [ ]:
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
df2.groupby(mapping).sum()

Die Gruppierung nach mehreren Spalten bildet einen hierarchischen Index.

In [ ]:
df2.groupby([mapping, "key"]).sum()

Video-Tutorial von Pycon 2015:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('5JnMutdy6Fw')

Hier ist das Übungsblatt zu diesem Notebook: [**06 - Übungsaufgaben Erstes Data Wrangling mit Pandas**](06_uebungsaufgaben_erstes_data_wrangling_mit_pandas.ipynb)


---

Wahlpflichtach Künstliche Intelligenz I: Praktikum